<a href="https://colab.research.google.com/github/ShreyaAN10/LearnDeepLearning/blob/main/PlotDecisionBoundaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This function is a utility for visualizing decision boundaries of a machine learning model trained on 2D features. It can handle both binary and multi-class classification problems. The decision boundaries are plotted on a meshgrid, and the input data points are also visualized on the same plot. The function relies on PyTorch for model inference and NumPy/Matplotlib for data manipulation and visualization.

## Imports

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np

1. Setup:
  - `model.to("cpu")`: Moves the model to the CPU. PyTorch models can be on either CPU or GPU, and this line ensures that the subsequent computations are performed on the CPU.
  - `X, y = X.to("cpu"), y.to("cpu")`: Moves the input data tensors `X` and `y` to the CPU as well. This is done to ensure consistency between PyTorch and NumPy operations.
2. Grid Creation:
  - Computes the minimum and maximum values of the features in X and adds a small margin `(0.1)` to create a boundary.
  - Creates a 2D meshgrid `(xx, yy)` using np.meshgrid based on the minimum and maximum values of the features. This grid is used to generate points for predictions across the entire feature space.
3. Features Creation:
  - `np.column_stack((xx.ravel(), yy.ravel()))`: Flattens and stacks the xx and yy meshgrid arrays to create a 2D array of feature points.
  - `torch.from_numpy(...)`: Converts the NumPy array to a PyTorch tensor.
  - `.float()`: Converts the tensor to a float data type.

4. Model Prediction:

  - `model.eval()`: Sets the model to evaluation mode. This is important when there are layers like dropout or batch normalization that behave differently during training and evaluation.
  - `with torch.inference_mode():`: Sets the model to inference mode. This is a context manager that temporarily disables gradient computation and reduces memory consumption.
  - `y_logits = model(X_to_pred_on)`: Obtains the logits (raw scores) for the generated feature points.

5. Prediction Processing:

  - Checks whether the problem is binary or multi-class based on the number of unique classes in y.
    - For multi-class classification: `torch.softmax(y_logits, dim=1).argmax(dim=1)`: Applies softmax to the logits and retrieves the class with the highest probability as the predicted class.
    - For binary classification:
`torch.round(torch.sigmoid(y_logits))`: Applies sigmoid activation to the logits and rounds to the nearest integer to obtain binary predictions (0 or 1).


6. Plotting:
- Reshapes the predicted labels to match the shape of the meshgrid `(xx and yy)`.
  - `plt.contourf(xx, yy, y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)`: Plots filled contours based on the reshaped predictions, creating decision boundaries.
  - `plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)`: Plots the input data points, where color (c) represents the true labels y.
  - `plt.xlim(xx.min(), xx.max()) and plt.ylim(yy.min(), yy.max())`: Sets the limits of the plot to cover the entire meshgrid.-


In [3]:
def plot_decision_boundaries(model: torch.nn.Module,
                             X:     torch.Tensor,
                             y:     torch.Tensor):
  """
  Plot decision boundaries of a machine learning model on a 2D feature space
  """

  # Setup
  device = "cpu"
  model.to(device)
  X, y = X.to(device), y.to(device)

  # Grid Creation
  x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
  y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
  xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))

  # Features Creation
  X_to_pred_on = torch.from_numpy(np.column_stack((xx.ravel(), yy.ravel()))).float()

  # Model Prediction
  model.eval()
  with torch.inference_mode():
    y_logits = model(X_to_pred_on)

  # Prediction Processing
  # Test for multi-class or binary and adjust logits to prediction labels
  if len(torch.unique(y)) > 2:
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)  # mutli-class
  else:
    y_pred = torch.round(torch.sigmoid(y_logits))  # binary

  # Plotting
  y_pred = y_pred.reshape(xx.shape).detach().numpy()
  plt.contourf(xx, yy, y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)
  plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
  plt.xlim(xx.min(), xx.max())
  plt.ylim(yy.min(), yy.max())